In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

We have created a small CSV file, Baby_Dataset_3.csv, which contains 4 books/authors with their checkouts. The goal is to count the number of checkouts in the first year, calculated as the checkouts between the first checkout date until one year after that. 

In [7]:
#Read in the sample dataset file 
df = pd.read_csv('data/Baby_Dataset_3.csv')
#Cast the CheckoutDate column as a DateTime for processing later
df['CheckoutDate'] = pd.to_datetime(df['CheckoutDate'])

In [11]:
books_john = df[(df['Title'] == 'The Collector') & (df['Creator'] == 'John Fowles')]
books_john = books_john.sort_values(by='CheckoutDate')
books_john.head(15)

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate
268,Digital,OverDrive,EBOOK,2013,5,1,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-05-01
269,Digital,OverDrive,EBOOK,2013,6,3,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-06-01
270,Digital,OverDrive,EBOOK,2013,7,6,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-07-01
271,Digital,OverDrive,EBOOK,2013,8,3,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-08-01
272,Digital,OverDrive,EBOOK,2013,9,2,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-09-01
273,Digital,OverDrive,EBOOK,2013,10,2,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-10-01
274,Digital,OverDrive,EBOOK,2013,11,3,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-11-01
275,Digital,OverDrive,EBOOK,2013,12,1,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-12-01
276,Digital,OverDrive,EBOOK,2014,4,1,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2014-04-01
277,Digital,OverDrive,EBOOK,2014,5,3,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2014-05-01


In [8]:
#Get unique pairs of (Title, Creator) to loop through
pairs = df[['Title', 'Creator']].value_counts().reset_index()
pairs = pairs.drop(columns='count')

#This prints the (Title, Creator) pairs
for index, row in pairs.iterrows():
    print(index)
    print(row)



0
Title      The Collector
Creator      John Fowles
Name: 0, dtype: object
1
Title      The Collector
Creator     Nora Roberts
Name: 1, dtype: object
2
Title          22/11/63
Creator    Stephen King
Name: 2, dtype: object
3
Title            The Game
Creator    Elizabeth Neal
Name: 3, dtype: object


In [5]:
column_names = df.columns.to_list()
column_names.append('CheckoutsFirstYear')
data = []

for index, row in pairs.iterrows():
    books = df[(df['Title'] == row.Title) & (df['Creator'] == row.Creator)]
    books = books.sort_values(by='CheckoutDate')
    row_new = books.iloc[0].to_list()
    first_checkout = books.iloc[0]['CheckoutDate']
    end_year = first_checkout.replace(year=first_checkout.year +1)
    sum_checkouts = books[books['CheckoutDate'] < end_year].Checkouts.sum()
    row_new.append(sum_checkouts)
    data.append(row_new)

df_sum = pd.DataFrame(data, columns = column_names)
df_sum.head()
    

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate,CheckoutsFirstYear
0,Digital,OverDrive,EBOOK,2013,5,1,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-05-01,22
1,Digital,OverDrive,EBOOK,2014,4,10,The Collector,Nora Roberts,"Fiction, Literature, Romance, Suspense","Penguin Group (USA), Inc.",2014,2014-04-01,329
2,Digital,OverDrive,EBOOK,2017,7,4,22/11/63,Stephen King,"Fiction, Horror, Literature","Penguin Random House Grupo Editorial, S.A.U.",2017,2017-07-01,42
3,Digital,OverDrive,EBOOK,2021,4,2,The Game,Elizabeth Neal,"Fantasy, Young Adult Fiction, Young Adult Lite...",Lerner Publishing Group,2021,2021-04-01,17


Here we have the sum of the checkouts from the first year in the CheckoutsFirstYear column. Note that the date in the CheckoutDate column gives the first checkout date, from which we calculate the first year. I have checked this manually for these 4 books. 

In [6]:
#Dropping the redundant columns
df_sum = df_sum.drop(columns=['CheckoutYear', 'CheckoutMonth', 'Checkouts'])
df_sum.head()

,UsageClass,CheckoutType,MaterialType,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate,CheckoutsFirstYear
0,Digital,OverDrive,EBOOK,The Collector,John Fowles,"Classic Literature, Fiction, Literature","Hachette Digital, Inc.",2013,2013-05-01,22
1,Digital,OverDrive,EBOOK,The Collector,Nora Roberts,"Fiction, Literature, Romance, Suspense","Penguin Group (USA), Inc.",2014,2014-04-01,329
2,Digital,OverDrive,EBOOK,22/11/63,Stephen King,"Fiction, Horror, Literature","Penguin Random House Grupo Editorial, S.A.U.",2017,2017-07-01,42
3,Digital,OverDrive,EBOOK,The Game,Elizabeth Neal,"Fantasy, Young Adult Fiction, Young Adult Lite...",Lerner Publishing Group,2021,2021-04-01,17
